In [1]:
from google.cloud import bigquery
import os
import pandas as pd

# https://docs.opensource.observer/docs/get-started/
# add GCP project and credentials here

PROJECT = 'opensource-observer'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../gcp_credentials.json'
client = bigquery.Client()

In [2]:
query = f"""
select
    project_name,
    display_name
from `{PROJECT}.oso.projects_v1`
"""
results = client.query(query)
projects = results.to_dataframe()

In [26]:
DATA_DIR = '../data'
dataframes = []

for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        if file.lower().endswith('.csv'):
            file_path = os.path.join(root, file)
            try:
                csv = pd.read_csv(file_path)
                csv['file_path'] = file_path
                dataframes.append(csv)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
else:
    df = pd.DataFrame()
    print("No CSV files found.")


In [27]:
list_projects = df['to_project_name'].dropna().unique()
len(list_projects)

649

In [28]:
for p in list_projects:
    if p not in projects['project_name'].unique():
        print(p)

In [29]:
df[df['to_project_name'] == 'scope-lift']

,to_project_name,amount,funding_date,from_funder_name,grant_pool_name,metadata,file_path


In [1]:
import pandas as pd
import yaml
import json

In [16]:
DIR = "../data/funders/dao_drops/"

In [17]:
COLS = [
    "oso_slug",
    "project_name",
    "project_id",
    "project_url",
    "project_address",
    "funder_name",
    "funder_round_name",
    "funder_round_type",
    "funder_address",
    "funding_amount",
    "funding_currency",
    "funding_network",
    "funding_date"
]
prices = {
    ('ARB', '2023-12-01'): 1.75,
    ('ETH', '2023-10-19'): 2000,
    ('ETH', '2024-01-17'): 2500,
    ('ETH', '2024-04-30'): 3010,    
    ( 'OP', '2022-07-01'): 1.00,
    ( 'OP', '2022-11-01'): 1.25,
    ( 'OP', '2023-01-01'): 1.25,
    ( 'OP', '2023-02-01'): 1.50,
    ( 'OP', '2023-04-01'): 1.50,
    ( 'OP', '2023-05-01'): 1.50,
    ( 'OP', '2023-09-01'): 1.75,
    ( 'OP', '2024-01-01'): 3.50,
    ( 'OP', '2024-07-17'): 1.80,
}    

In [18]:
def datify(x):

    if isinstance(x, str):
        if "/" in x:
            date = pd.to_datetime(x, dayfirst=False)
        elif "T" in x:
            x = x.split("T")[0]
            date = pd.to_datetime(x)
        elif ":" in x:
            x = x.split(" ")[0]
            date = pd.to_datetime(x)
        else:
            date = pd.to_datetime(x)
    else:
        date = pd.to_datetime(x.date())
    string = f"{date.year}-{str(date.month).zfill(2)}-{str(date.day).zfill(2)}"
    return string
    
def price(currency, date, amt):
    if pd.isna(amt):
        return 0
    if currency in ['DAI', 'USD']:
        return amt
    else:
        fx = prices.get((currency, date))
        return amt * fx

In [19]:
dao_drops = pd.read_csv(DIR + "dao_drops_grants.csv", index_col=0)

dao_drops.rename(columns={
    "Project": "project_name",
    "To": "project_address",
    "DAI": "funding_amount",
}, inplace=True)

dao_drops["project_id"] = None
dao_drops["project_url"] = "https://daodrops.io/"

dao_drops["funder_name"] = "DAO Drops (dOrg)"
dao_drops["funder_address"] = "0xafe5f7a1d1c173b311047cdc93729013ad03de0c"
dao_drops["funding_currency"] = "DAI"
dao_drops["funding_network"] = "mainnet"
dao_drops["funder_round_type"] = "Retroactive"
dao_drops["funder_round_name"] = "Round 1"
dao_drops["funding_date"] = pd.to_datetime("2023-03-29")

temp = dao_drops[COLS].copy()

temp['funding_date'] = temp['funding_date'].apply(datify)
temp['funding_usd'] = temp.apply(
    lambda x: price(x['funding_currency'], x['funding_date'], x['funding_amount']), axis=1)

temp

,oso_slug,project_name,project_id,project_url,project_address,funder_name,funder_round_name,funder_round_type,funder_address,funding_amount,funding_currency,funding_network,funding_date,funding_usd
0,giveth,Giveth,None,https://daodrops.io/,0x4d9339dd97db55e3b9bcbe65de39ff9c04d1c2cd,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,18186,DAI,mainnet,2023-03-29,18186
1,NaN,DeFi Library,None,https://daodrops.io/,0x4e3d9399c28c724293dd545225ec843ac4c9e953,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,14373,DAI,mainnet,2023-03-29,14373
2,ethers-io,ethers.js,None,https://daodrops.io/,0x643aa0a61eadcc9cc202d1915d942d35d005400c,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,11801,DAI,mainnet,2023-03-29,11801
3,metagame-metafam,Metagame,None,https://daodrops.io/,0xb53b0255895c4f9e3a185e484e5b674bccfbc076,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,11287,DAI,mainnet,2023-03-29,11287
4,rotki,Rotki,None,https://daodrops.io/,0x9531c059098e3d194ff87febb587ab07b30b1306,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,10863,DAI,mainnet,2023-03-29,10863
5,eth-limo,eth.limo,None,https://daodrops.io/,0xb352bb4e2a4f27683435f153a259f1b207218b1b,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,10530,DAI,mainnet,2023-03-29,10530
6,gooddollar,GoodDollar,None,https://daodrops.io/,0xc9d09ba972b4eb6bb7347f2229fb6d50434ffca6,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,9199,DAI,mainnet,2023-03-29,9199
7,l2planet,L2 Planet,None,https://daodrops.io/,0x2d9d84f4d8a4254ea23fe18825193cbd1b21b749,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,9078,DAI,mainnet,2023-03-29,9078
8,gashawk-corpus-io,GasHawk,None,https://daodrops.io/,0x44f00c29a0b8514d017ac20fbdb0aa5078b860ce,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,8654,DAI,mainnet,2023-03-29,8654
9,NaN,LauNaMu,None,https://daodrops.io/,0xa565ea02caef7d1b6b5b919f3fcf829647ed50dd,DAO Drops (dOrg),Round 1,Retroactive,0xafe5f7a1d1c173b311047cdc93729013ad03de0c,7898,DAI,mainnet,2023-03-29,7898


In [20]:
df = temp[['funding_usd', 'funding_date', 'funder_name', 'oso_slug', 'funder_round_name']].copy()

df.rename(columns={
    'funder_name': 'from_funder_name',
    'oso_slug': 'to_project_name',
    'funder_round_name': 'grant_pool_name',
    'funding_usd': 'amount'
}, inplace=True)

#df['to_project_name'] = df['to_project_name'].fillna('na') 
df['from_funder_name'] = df['from_funder_name'].apply(lambda x: x.lower().replace(' ','_'))
df['grant_pool_name'] = df['grant_pool_name'].apply(lambda x: x.lower().replace(' ','_'))

def make_metadata(data):
    metadata = {
        'application_name': data['project_name'],
        'application_url': data['project_url'],
        'token_amount': data['funding_amount'],
        'token_unit': data['funding_currency']
    }
    return json.dumps(metadata)
    
    

df['metadata'] = temp.apply(make_metadata, axis=1)

for x in df['grant_pool_name'].unique():
    print("-",x)

- round_1


- funding_date
- from_funder_name
- to_project_name
- grant_pool_name
- amount_usd
- metadata

In [21]:
for x in df['grant_pool_name'].unique():
    dff = df[df['grant_pool_name'] == x].set_index('to_project_name', drop=True)
    dff.sort_index().to_csv(f"{DIR}uploads/{x}.csv")